# Michelson tutorial
## Chapter one
This chapter is an introduction to Michelson smart contracts structure and conventions.

##### 1. Understanding the stack

Michelson is a *low-level* programming language, which means that it doesn't offer all the features of *high-level* programming languages you may be used to, like JavaScript or C++. For example, there are no variables in Michelson.  
Every time you write code in Michelson, you must keep in mind something called "**the stack**". You can imagine the stack as a game of Jenga: it is a collection of elements that you pile up on top of each other, the goal of "Michelson Jenga" being to remove all the pieces except for a specific one that must remain at the end. In order to play, you will be given a list of *instructions* that can manipulate the pieces of the game: you can remove them, change their position, add some more on top, check if two pieces are the same to remove them, etc., but one false move and the stack collapses (which we call *failing*).

##### 2. Structure of a Smart Contract

A smart contract in Michelson displays a simple structure with 3 parts:
1. A part indicating the type of the storage
2. A part indicating the type of the parameter
3. The code to be executed on incoming transactions  
_(Note that the order doesn't matter)_

For every incoming transaction, there will be only one parameter. However, the parameter can be, for example, a pair and contain 2 elements. Likewise, the storage is made of a single variable, which can also be a pair, a map, etc. and contain multiple values.

Here is an example of a very simple smart contract (which doesn't modify its storage):

In [1]:
storage unit ;
parameter unit ;
code {
    DROP ;
    PUSH unit Unit ;
    NIL operation ;
    PAIR
}

storage unit;
parameter unit;
code { DROP ; PUSH unit Unit ; NIL operation ; PAIR };

The smart contract removes the pair containing the parameter and the storage from the top of the stack, pushes a unit, an empty list of operation and pair them before returning them.

A smart contract in Michelson always has to return a pair containing a list of operations to be run at the end of the execution and the new storage.

When you are ready to run the smart contract, you can open a new cell and type the following command:

In [2]:
RUN %default Unit Unit ;

RUN: use %default; drop all; push (Unit, Unit);
  DROP: pop (Unit, Unit);
  PUSH: push Unit;
  NIL: push [];
  PAIR: pop [], Unit; push ([], Unit);

value,type
Unit,unit


**RUN** starts the execution of the smart contract, **%default** targets the default entry point of the smart contract (the only one there is at the moment), the first **Unit** is the parameter and the second **Unit** is the storage.

##### 3. Operation codes

The *code* part of the Michelson contract consists of a serie of instructions that the contract will perform every time it is called. These instructions are run top to bottom and may or may not be followed by parameters. In general, for the operation codes that accept parameters, these parameters indicate the position of the stack to manipulate, the type of value associated to the newly created element or the value itself.  
Note that every instruction ends when a semi-colon is inserted (however, you can omit it for the last instruction (generally `PAIR`).

Let's have a look at the instructions introduced earlier:

`DROP`: this instruction gets rid of the first element on top of the stack.

`PUSH`: this instruction is the opposite of the `DROP` instruction and adds an element on top of the stack.   
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;You must specify the type and the value, for example `PUSH string "hello"` or `PUSH int 3`.

`NIL`: like `PUSH`, `NIL` introduces a new element on top of the stack. This element is a list.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;After the opcode, you must specify the type of values the list will contain (lists can only contain one single type).  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;As we need a list of operations to end the execution of the smart contract, we add an empty list of operations.

`PAIR`: finally, as the contract is expecting a pair containing a list of instructions and the new storage, we must create this pair.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The `PAIR` opcode takes the two elements on top of the stack and put them together in a pair.  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;The first one will be in the left position of the pair, the second one in the right position.

##### 4. Running the contract step-by-step

This notebook allows you to run the smart contract step-by-step and visualize the state of the stack at every step!

In order to use this feature, there are 3 instructions you must remember. These instructions are only used in the context of these notebooks and are NOT Michelson instructions:

- `BEGIN`: use it to start the execution of the contract. You must also specify the parameter and the storage as in : `BEGIN param storage`.  
- `DUMP`: add it to display the current state of the stack.  
- `COMMIT`: use it to end the execution of the contract and return the list/storage pair.

Let's rewrite the example above and use these three instructions:

In [15]:
storage unit ;
parameter unit ;

storage unit;
parameter unit;

In [16]:
BEGIN Unit Unit ;

value,type
Pair Unit Unit,pair unit unit


In [17]:
DROP ;

In [18]:
DUMP ; ## The stack is empty now as we removed the only element that was there

stack is empty

In [19]:
PUSH unit Unit ;

value,type
Unit,unit


In [20]:
NIL operation ;

value,type
[],list operation


In [21]:
DUMP ;

value,type
[],list operation
Unit,unit


In [22]:
PAIR ;

value,type
Pair {} Unit,pair (list operation) unit


In [23]:
COMMIT ;

value,type
Unit,unit
